In [1]:
# !pip install keras
import numpy as np
import pandas as pd
import os
from sklearn.metrics import mean_squared_error
import time
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, GRU, Bidirectional

In [2]:
dataset = pd.read_csv("../input/stockprice/stockprice.csv",index_col='Date', parse_dates=['Date'])

In [3]:
dataset.shape

In [4]:
dataset.head()

In [5]:
train_set = dataset[:'2016'].iloc[:, 1:2].values # High value from 2006 to 2016
test_set = dataset['2017':].iloc[:,1:2].values # High value in 2017

In [6]:
train_set.shape

In [7]:
test_set.shape

In [8]:
def pic(test_result, predict_restult, title):
    plt.plot(test_result, color='red', label='Ground Truth')
    plt.plot(predict_restult, color='blue', label='Prediction')
    plt.title(title)
    plt.xlabel("Day")
    plt.ylabel("Stock Price")
    plt.legend()
    plt.show()

In [9]:
dataset['High'][:"2016"].plot(figsize=(16, 4), legend=True)
dataset['High']["2017":].plot(figsize=(16, 4), legend=True)
plt.title("Overview of Data")
plt.legend(['Training data', 'Testing data'])
plt.show()

In [10]:
sc = MinMaxScaler(feature_range=[0, 1])
train_set_scaled = sc.fit_transform(train_set)

In [11]:
# 60 days a sample, one output
X_train = []
y_train = []
for i in range(60, 2769):
    X_train.append(train_set_scaled[i-60:i, 0]) 
    y_train.append(train_set_scaled[i, 0])
X_train, y_train = np.array(X_train), np.array(y_train) 

In [12]:
X_train.shape

In [13]:
# LSTM input(samples, sequence_length, features) 
# reshape: training set(2709,60)  ---> (2709, 60, 1)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1)) 

In [14]:
X_train.shape

In [15]:
lstm_model = Sequential()
# LSTM 1st layer
lstm_model.add(LSTM(128, return_sequences=True, input_shape=(X_train.shape[1], 1))) # input shape is 60
lstm_model.add(Dropout(0.2))

# LSTM 2nd layer
lstm_model.add(LSTM(128, return_sequences=True))
lstm_model.add(Dropout(0.2))

# LSTM 3th layer
lstm_model.add(LSTM(128)) # the default return_sequence is false, therefore, it will return 1 deminsion
lstm_model.add(Dropout(0.2))

# Dense Layer
lstm_model.add(Dense(units=1))

In [16]:
# compile the model
lstm_model.compile(optimizer='rmsprop', loss='mse')

# train the model
time_start = time.time()  # Record start time
lstm_model.fit(X_train, y_train, epochs=20, batch_size=32)
time_end = time.time()  # Record finish time
time_sum = time_end - time_start  # Unit is second
print("Time for training LSTM model is: ",time_sum," second!")

In [17]:
eporch=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
loss=[0.0280,0.0096,0.0071,0.0055,0.0051,0.0045,0.0037,0.0034,0.0034,0.0029,0.0029,0.0027,0.0026,0.0026,0.0023,0.0022,0.0021,0.0019,0.0019,0.0018]
new_ticks = np.linspace(1, 20, 20)

plt.xticks(new_ticks)
plt.plot(loss)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Relationship Between Epoch and Loss')

In [18]:
dataset_total = pd.concat((dataset['High'][:"2016"], dataset['High']["2017":]), axis=0)

In [19]:
dataset_total.shape

In [20]:
dataset_total

In [21]:
inputs = dataset_total[len(train_set):].values

In [22]:
inputs = inputs.reshape(-1, 1)

In [23]:
inputs.shape

In [24]:
inputs_scaled = sc.fit_transform(inputs)

In [25]:
dataset_total = pd.concat((dataset['High'][:"2016"], dataset['High']["2017":]), axis=0)
# input data
inputs = dataset_total[len(dataset_total) - len(test_set) - 60:].values

In [26]:
# normalization
inputs = inputs.reshape(-1, 1)
inputs = sc.transform(inputs)

In [27]:
inputs.shape

In [28]:
# X_test for training
X_test = []
for i in range(60, 311):
    X_test.append(inputs[i-60:i, 0])
    
X_test = np.array(X_test)

X_test.shape

In [29]:
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

In [30]:
X_test.shape

In [31]:
predict_test = lstm_model.predict(X_test) # Predict

In [32]:
predict_test.shape

In [33]:
predict_stock_price = sc.inverse_transform(predict_test)  # Inverse transform, otherwise, the result is in range 0-1

In [34]:
print(predict_stock_price)
print(predict_stock_price.shape)

In [35]:
# Visualization
pic(test_set, predict_stock_price,"LSTM_Model")

In [36]:
# Calculate MSE of LSTM model's result
LSTM_MSE = mean_squared_error(test_set,predict_stock_price)
print("MSE of LSTM is: ",LSTM_MSE)

### **GRU Model**

In [37]:
gru_model = Sequential()
gru_model.add(GRU(128, return_sequences=True, input_shape=(X_train.shape[1], 1), activation='tanh'))
gru_model.add(Dropout(0.2))
gru_model.add(GRU(128, return_sequences=True))
gru_model.add(Dropout(0.2))
gru_model.add(GRU(128, activation='tanh'))
gru_model.add(Dropout(0.2))
gru_model.add(Dense(1))
# Compile
gru_model.compile(optimizer='rmsprop', loss='mse')
# Train
time_start = time.time()  # Record start time
gru_model.fit(X_train, y_train, epochs=20, batch_size=32)
time_end = time.time()  # Record finish time
time_sum = time_end - time_start  # Unit is s
print("Time for training GRU model is:",time_sum," second!")
# Prepare testing set
X_test = []
for i in range(60, 311):
    X_test.append(inputs[i-60:i, 0])
    
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
GRU_predicted = gru_model.predict(X_test)
print(GRU_predicted)
print(GRU_predicted.shape)
GRU_predicted_stock_price = sc.inverse_transform(GRU_predicted)
pic(test_set, GRU_predicted_stock_price,"GRU_Model")

In [38]:
#Calculate MSE of GRU model's result
GRU_MSE = mean_squared_error(test_set,GRU_predicted_stock_price)
print("MSE of GRU is: ",GRU_MSE)

### **Bi-LSTM Model**

In [39]:
bilstm_model = Sequential()
bilstm_model.add(Bidirectional(LSTM(128, return_sequences=True), input_shape=(X_train.shape[1], 1)))
bilstm_model.add(Dropout(0.2))
bilstm_model.add(Bidirectional(LSTM(64, return_sequences=True)))
bilstm_model.add(Dropout(0.2))
bilstm_model.add(Bidirectional(LSTM(64)))
bilstm_model.add(Dropout(0.2))
bilstm_model.add(Dense(1))
# Compile the model
bilstm_model.compile(optimizer='rmsprop', loss='mse')
# Training
time_start = time.time()  
bilstm_model.fit(X_train, y_train, epochs=20, batch_size=32)
time_end = time.time()  
time_sum = time_end - time_start  #Unit is s
print(time_sum)

X_test = []
for i in range(60, 311):
    X_test.append(inputs[i-60:i, 0])
    
X_test = np.array(X_test) 

X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
bilstm_predicted = bilstm_model.predict(X_test)
print(bilstm_predicted)
print(bilstm_predicted.shape)
bilstm_predicted_stock_price = sc.inverse_transform(bilstm_predicted)

pic(test_set, bilstm_predicted_stock_price,"Bi-LSTM Model")

In [40]:
bilstm_MSE = mean_squared_error(test_set,bilstm_predicted_stock_price)
print("MSE of Bi-LSTM is: ",bilstm_MSE)